# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [8]:
# !pip install langchain
# !pip install -qU langchain-groq
import os

# Load a specific environment variable
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [9]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
#!pip install pandas

## LLMChain

In [22]:
# !pip install langchain --upgrade
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
# from langchain import tool, LLMChain
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [24]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [14]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [29]:
# chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm | output_parser

In [30]:
product = "Queen Size Sheet Set"
# chain.run(product)
chain.invoke({"product":"Queen Size Sheet Set"})

'What a great question! Here are some suggestions for a company name that makes Queen Size Sheet Set:\n\n1. **Royal Bedding Co.**: This name plays off the idea of a Queen-sized bed being a luxurious and regal experience.\n2. **Dreamweaver Sheets**: This name evokes a sense of comfort and coziness, and the word "weaver" suggests a high-quality, handmade product.\n3. **Queenly Linens**: This name incorporates the idea of a Queen-sized bed and adds a touch of elegance with "linens".\n4. **Slumber & Co.**: This name conveys a sense of relaxation and comfort, and the "& Co." suggests a company that specializes in bedding.\n5. **Soft & Serene**: This name emphasizes the softness and serenity of the sheets, which is a key selling point for a product like this.\n6. **Bedding Bliss**: This name suggests that the sheets will bring joy and happiness to those who use them.\n7. **Luxe Bedding Co.**: This name emphasizes the luxurious quality of the sheets and suggests a high-end product.\n8. **Rest

## SimpleSequentialChain

In [34]:
from langchain.chains import SimpleSequentialChain

In [31]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [32]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [35]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [36]:
overall_simple_chain.run(product)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
What a great question!

Naming a company can be a fun and creative process. Here are some suggestions for a company that makes Queen-size sheet sets:

1. **Royal Linens**: This name plays off the idea of the Queen-size bed being a royal treatment, and "Linens" is a clear indication of what the company produces.
2. **Queenly Bedding**: This name incorporates the idea of the Queen-size bed and emphasizes the high-quality bedding produced by the company.
3. **ComfortCraft Co.**: This name highlights the company's focus on creating comfortable bedding products, and "Co." suggests a company that crafts its products with care.
4. **DreamWeave**: This name evokes the idea of a cozy, dreamy sleeping experience, and "Weave" references the manufacturing process of creating sheet sets.
5. **Bedding Bliss**: This name conveys a sense of happiness and satisfaction, suggesting that the company's sheet sets will bring joy to those who sleep in them.
6. 

'Here is a 20-word description for the company:\n\n"Royal Linens: Crafting luxurious Queen-size sheet sets with comfort, quality, and style, ensuring a restful night\'s sleep for all."'

## SequentialChain

In [37]:
from langchain.chains import SequentialChain

In [38]:
# llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [39]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [40]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [41]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [42]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [43]:
review = """Hertz, fácil de encontrar con excelente servicio. Alquilamos un Auto DS (Citroen), este es un vehículo excepcional y es un placer conducirlo. El mejor sistema de navegación que he usado y perfecto. El precio era un poco alto pero valió la pena.
Cliente feliz."""
overall_chain(review)

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Hertz, fácil de encontrar con excelente servicio. Alquilamos un Auto DS (Citroen), este es un vehículo excepcional y es un placer conducirlo. El mejor sistema de navegación que he usado y perfecto. El precio era un poco alto pero valió la pena.\nCliente feliz.',
 'English_Review': 'Here is the translation:\n\n"Hertz, easy to find with excellent service. We rented a Auto DS (Citroen), this is an exceptional vehicle and it\'s a pleasure to drive it. The best navigation system I\'ve ever used and perfect. The price was a bit high, but it was worth it.\nHappy customer."\n\nLet me know if you have any other requests!',
 'summary': 'Here is a summary of the review in 1 sentence:\n\nThe reviewer had a positive experience with Hertz, praising the ease of finding the location, excellent service, and exceptional vehicle (Citroen Auto DS) with a perfect navigation system, despite the slightly high price.',
 'followup_message': '¡Claro! Aquí está un seguimiento de la respuesta en españ

## Router Chain

In [44]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [45]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
# llm = ChatOpenAI(temperature=0, model=llm_model)

In [50]:
from pprint import pprint
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
pprint(destinations_str)

('physics: Good for answering questions about physics\n'
 'math: Good for answering math questions\n'
 'History: Good for answering history questions\n'
 'computer science: Good for answering computer science questions')


In [51]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [52]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [53]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [54]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [55]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of thermodynamics?'}
> Finished chain.


"Black body radiation is a fundamental concept in thermodynamics, and it's a fascinating topic.\n\nIn simple terms, black body radiation refers to the emission of energy by an object in the form of electromagnetic radiation, such as light, when it is heated to a certain temperature. This radiation is emitted by the object itself, rather than being reflected or transmitted by it.\n\nIn the context of thermodynamics, black body radiation is important because it's a way for objects to release excess energy they've gained from being heated. When an object is heated, its molecules start moving faster and gaining kinetic energy. Some of this energy is converted into thermal energy, which is the energy associated with the motion of the molecules. However, as the object continues to heat up, it reaches a point where it can no longer absorb any more energy without emitting it as radiation. This is where black body radiation comes in.\n\nAt a given temperature, a black body will emit radiation a

In [56]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'A simple question, but one that requires a bit of finesse nonetheless!\n\nTo tackle this question, I\'ll break it down into its component parts:\n\n1. What is the definition of the operation "+", also known as addition?\n\nIn mathematics, addition is defined as the operation of combining two or more numbers to produce a sum. In this case, we\'re combining two numbers, 2 and 2.\n\n2. What are the properties of the numbers 2 and 2?\n\nBoth 2 and 2 are integers, which are a type of whole number. They are also equal in value, which means they have the same magnitude and opposite signs (since they\'re both positive).\n\n3. How do we apply the definition of addition to these numbers?\n\nAccording to the definition, we combine the two numbers by adding their values together. In this case, we\'re adding 2 + 2.\n\nNow, let\'s calculate the sum:\n\n2 + 2 = ?\n\nTo do this, we can count the total number of units:\n\n2 units (from the first 2) + 2 units (from the second 2) = 4 units\n\nTherefore,

In [36]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Why does every cell in our body contain DNA, specifically?'}
> Finished chain.


'What a great question!\n\nThe answer lies in the fundamental concept of heredity and the mechanisms of biological inheritance. You see, DNA (Deoxyribonucleic acid) is the molecule that contains the genetic instructions used in the development and function of all living organisms. It\'s often referred to as the "blueprint" or "instructions" for life.\n\nIn the case of cells, DNA serves as the primary repository of genetic information, which is crucial for the cell\'s survival, growth, and reproduction. The DNA molecule is made up of two complementary strands that are twisted together to form a double helix structure. This structure is incredibly compact, allowing a vast amount of genetic information to be stored in a relatively small space.\n\nNow, every cell in our body contains DNA because it\'s essential for the cell\'s function. DNA provides the instructions for the cell to synthesize proteins, which are the building blocks of all biological structures and processes. It also contai

Reminder: Download your notebook to you local computer to save your work.

In [37]:
await llm.ainvoke(messages)

NameError: name 'messages' is not defined